##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Save and load Keras models

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/save_and_serialize"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/snapshot-keras/site/en/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/keras-team/keras-io/blob/master/guides/serialization_and_saving.py"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Introduction

A Keras model consists of multiple components:

- The architecture, or configuration, which specifies what layers the model
contain, and how they're connected.
- A set of weights values (the "state of the model").
- An optimizer (defined by compiling the model).
- A set of losses and metrics (defined by compiling the model or calling
`add_loss()` or `add_metric()`).

The Keras API makes it possible to save all of these pieces to disk at once,
or to only selectively save some of them:

- Saving everything into a single archive in the TensorFlow SavedModel format
(or in the older Keras H5 format). This is the standard practice.
- Saving the architecture / configuration only, typically as a JSON file.
- Saving the weights values only. This is generally used when training the model.

Let's take a look at each of these options. When would you use one or the other,
and how do they work?

## How to save and load a model

If you only have 10 seconds to read this guide, here's what you need to know.

**Saving a Keras model:**

```python
model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save('path/to/location')
```

**Loading the model back:**

```python
from tensorflow import keras
model = keras.models.load_model('path/to/location')
```

Now, let's look at the details.

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Whole-model saving & loading

You can save an entire model to a single artifact. It will include:

- The model's architecture/config
- The model's weight values (which were learned during training)
- The model's compilation information (if `compile()` was called)
- The optimizer and its state, if any (this enables you to restart training
where you left)

#### APIs

- `model.save()` or `tf.keras.models.save_model()`
- `tf.keras.models.load_model()`

There are two formats you can use to save an entire model to disk:
**the TensorFlow SavedModel format**, and the older Keras **H5 format**.
The recommended format is SavedModel. It is the default when you use `model.save()`.

You can switch to the H5 format by:

- Passing `save_format='h5'` to `save()`.
- Passing a filename that ends in `.h5` or `.keras` to `save()`.

### SavedModel format

SavedModel is the more comprehensive save format that saves the model architecture,
weights, and the traced Tensorflow subgraphs of the call functions. This enables
Keras to restore both built-in layers as well as custom objects.

**Example:**

In [2]:
def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


In [3]:
model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

4/4 [==============================] - 1s 4ms/step - loss: 0.2868


In [4]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model")

In [5]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_model")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 3ms/step - loss: 0.2493


#### What the SavedModel contains

Calling `model.save('my_model')` creates a folder named `my_model`,
containing the following:

In [6]:
!ls -al my_model

total 72
drwxr-xr-x 4 root root  4096 Oct 13 00:16 .
drwxr-xr-x 1 root root  4096 Oct 13 00:16 ..
drwxr-xr-x 2 root root  4096 Oct 13 00:16 assets
-rw-r--r-- 1 root root  4647 Oct 13 00:16 keras_metadata.pb
-rw-r--r-- 1 root root 46273 Oct 13 00:16 saved_model.pb
drwxr-xr-x 2 root root  4096 Oct 13 00:16 variables


The model architecture, and <font color="green">training configuration
(including the optimizer, losses, and metrics)</font> are stored in `saved_model.pb`.
The weights are saved in the `variables/` directory.

For detailed information on the SavedModel format, see the
[SavedModel guide (*The SavedModel format on disk*)](
  https://www.tensorflow.org/guide/saved_model#the_savedmodel_format_on_disk).


#### How SavedModel handles custom objects

When saving the model and its layers, the SavedModel format stores the
class name, **call function**, losses, and weights (and the config, if implemented).
The **call function** defines the computation graph of the model/layer.

In the absence of the model/layer config, the call function is used to create
a model that exists like the original model which can be trained, evaluated,
and used for inference.

Nevertheless, it is always a good practice to define the `get_config`
and `from_config` methods when writing a custom model or layer class.
This allows you to easily update the computation later if needed.
See the section about [Custom objects](#custom-objects)
for more information.

Example:

모델과 해당 모델의 레이어를 저장할 때 SavedModel 형식은 클래스명, call 함수, losses 및 weights(구현된 경우에는 config도 포함)를 저장합니다. **call 함수는 모델/레이어의 계산 graph를 정의합니다.**

모델/레이어 config가 없는 경우 call 함수는 훈련, 평가 및 추론에 사용될 수 있는 원본 모델과 같은 모델을 만드는 데 사용됩니다.

그럼에도 불구하고 사용자 정의 model 또는 layer 클래스를 작성할 때 항상 get_config 및 from_config 메서드를 정의하는 것이 좋습니다. 이를 통해 필요한 경우 나중에 computation을 쉽게 업데이트할 수 있습니다. 자세한 내용은 사용자 정의 객체에 대한 섹션을 참조하세요.

In [15]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        print("called CustomModel Init\n")
        self.hidden_units = hidden_units
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        print("called call\n")
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x
    # keras.Model.get_config을 override[overwrite]함
    def get_config(self):
        print("called get_config \n")
        return {"hidden_units": self.hidden_units}

    # keras.Model.from_config을 override[overwrite]함
    @classmethod
    def from_config(cls, config):
        print("called from_config \n")
        return cls(**config)

In [11]:
def testdAPara(a,b,c):
  print(a,b,c)

In [12]:
p={'a':'1', 'b':'2', 'c':'3'}
testdAPara(*p)

a b c


In [13]:
testdAPara(**p)

1 2 3


In [16]:
model = CustomModel([16, 16, 10])
# Build the model by calling it
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)

#######
print("call model.save method")
model.save("my_model")

called CustomModel Init

called call

call model.save method
called call

called call

called call

called get_config 

called get_config 

called call

called get_config 

called get_config 



In [17]:
# Option 1: Load with the custom_object argument.
print("1.call load_model method \n")

loaded_1 = keras.models.load_model(
    "my_model", custom_objects={"CustomModel": CustomModel}
)


print("del load_model method  \n")

# Option 2: Load without the CustomModel class.

# Delete the custom-defined model class to ensure that the loader does not have
# access to it.
del CustomModel

1.call load_model method 

called from_config 

called CustomModel Init

del load_model method  



In [19]:
print("2.call load_model method  \n")
loaded_2 = keras.models.load_model("my_model")

np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

print("\n Original model:", model)
print("Model Loaded with custom objects:", loaded_1)
print("Model loaded without the custom object class:", loaded_2)

2.call load_model method  



called call


 Original model: <__main__.CustomModel object at 0x7fc3526d5ad0>
Model Loaded with custom objects: <__main__.CustomModel object at 0x7fc352052ed0>
Model loaded without the custom object class: <keras.saving.saved_model.load.CustomModel object at 0x7fc353480fd0>


In [20]:
print(loaded_2(input_arr))
print(outputs)

tf.Tensor(
[[ 0.684122    0.4062841   0.10979186  0.15771788  0.8472042  -1.0043093
   0.50487673 -0.04807676  0.62883776  0.08274942]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[ 0.684122    0.4062841   0.10979186  0.15771788  0.8472042  -1.0043093
   0.50487673 -0.04807676  0.62883776  0.08274942]], shape=(1, 10), dtype=float32)


The first loaded model is loaded using the config and `CustomModel` class. The second
model is loaded by dynamically creating the model class that acts like the original model.

#### Configuring the SavedModel

*New in TensoFlow 2.4*
The argument `save_traces` has been added to `model.save`, which allows you to toggle
SavedModel function tracing. Functions are saved to allow the Keras to re-load custom
objects without the original class definitons, so when `save_traces=False`, all custom
objects must have defined `get_config`/`from_config` methods. When loading, the custom
objects must be passed to the `custom_objects` argument. `save_traces=False` reduces the
disk space used by the SavedModel and saving time.

TensoFlow 2.4의 새로운 기능인 save_traces 인수가 model.save에 추가되어 SavedModel 함수 추적을 토글할 수 있습니다.</br>
Keras가 원래 클래스 정의 없이 custom object를 다시 로드할 수 있도록 Function들이 저장되므로 save_traces=False인 경우 모든 custom object에는 get_config/from_config 메서드가 정의되어 있어야 합니다.</br>
로드할 때 custom object는 custom_objects 인수에 전달되어야 합니다. save_traces=False는 SavedModel이 사용하는 디스크 공간을 줄이고 시간을 절약합니다.

### Keras H5 format

Keras also supports saving a single HDF5 file containing the model's architecture,
weights values, and `compile()` information.
It is a light-weight alternative to SavedModel.

**Example:**

In [21]:
model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("my_h5_model.h5")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_h5_model.h5")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 4ms/step - loss: 0.2446


#### Limitations

Compared to the SavedModel format, there are two things that don't
get included in the H5 file:

- **External losses & metrics** added via `model.add_loss()`
& `model.add_metric()` are not saved (unlike SavedModel).
If you have such losses & metrics on your model and you want to resume training,
you need to add these losses back yourself after loading the model.
Note that this does not apply to losses/metrics created *inside* layers via
`self.add_loss()` & `self.add_metric()`. As long as the layer gets loaded,
these losses & metrics are kept, since they are part of the `call` method of the layer.
- The **computation graph of custom objects** such as custom layers
is not included in the saved file. At loading time, Keras will need access
to the Python classes/functions of these objects in order to reconstruct the model.
See [Custom objects](#custom-objects).


* `model.add_loss()` 및 `model.add_metric()`을 통해 추가된 **External losses & metrics**은 SavedModel과 달리 저장되지 않습니다. 모델에 이러한 손실 및 메트릭이 있고 training을 재개하려면 모델을 로드한 후 이러한 손실을 다시 추가해야 합니다.</br>
이것은 `self.add_loss`() 및 `self.add_metric()`을 통해 layer 내부에서 생성된 손실/메트릭에는 적용되지 않습니다.</br>
레이어가 로드되는 한 이러한 손실 및 메트릭은 레이어 `call` 메서드의 일부이기 때문에 유지됩니다.

* cutom layers 등 custom objects의 computation graph는 저장 파일에 포함되지 않습니다. 로드 시 Keras는 모델을 재구성하기 위해 이러한 객체의 Python 클래스/함수에 액세스해야 합니다.

## Saving the architecture

The model's configuration (or architecture) specifies what layers the model
contains, and how these layers are connected*. If you have the configuration of a model,
then the model can be created with a freshly initialized state for the weights
and no compilation information.

*Note this only applies to models defined using the functional or Sequential apis
 not subclassed models.

모델의 config(또는 아키텍처)은 모델에 포함된 레이어와 이러한 레이어의 연결 방법*을 지정합니다. </br>
모델에 대한 config가 있는 경우 weights에 대해 새로 초기화된 상태로 컴파일 정보 없이 모델이 생성될 수 있습니다.

*이 기능은 Subclassed된 모델이 아닌 Functional 또는 Sequential API를 사용하여 정의된 모델에만 적용됩니다.

### Configuration of a Sequential model or Functional API model

These types of models are explicit graphs of layers: their configuration
is always available in a structured form.

#### APIs

- `get_config()` and `from_config()`
- `tf.keras.models.model_to_json()` and `tf.keras.models.model_from_json()`

#### `get_config()` and `from_config()`

Calling `config = model.get_config()` will return a Python dict containing
the configuration of the model. The same model can then be reconstructed via
`Sequential.from_config(config)` (for a `Sequential` model) or
`Model.from_config(config)` (for a Functional API model).

The same workflow also works for any serializable layer.

**Layer example:**

In [22]:
layer = keras.layers.Dense(3, activation="relu")

In [23]:
layer_config = layer.get_config()

In [25]:
layer.get_config()

{'name': 'dense_14',
 'trainable': True,
 'dtype': 'float32',
 'units': 3,
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [26]:
new_layer = keras.layers.Dense.from_config(layer_config)

In [27]:
new_layer.get_config()

{'name': 'dense_14',
 'trainable': True,
 'dtype': 'float32',
 'units': 3,
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

**Sequential model example:**

In [28]:
model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])

In [29]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_3'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_15',
    'trainable': True,
    'dtype': 'float32',
    'units': 1,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}}]}

In [30]:
config = model.get_config()
new_model = keras.Sequential.from_config(config)

**Functional model example:**

In [31]:
inputs = keras.Input((32,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
config = model.get_config()
new_model = keras.Model.from_config(config)

In [32]:
model.get_config()

{'name': 'model_2',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_4'},
   'name': 'input_4',
   'inbound_nodes': []},
  {'class_name': 'Dense',
   'config': {'name': 'dense_16',
    'trainable': True,
    'dtype': 'float32',
    'units': 1,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'dense_16',
   'inbound_nodes': [[['input_4', 0, 0, {}]]]}],
 'input_layers': [['input_4', 0, 0]],
 'output_layers': [['dense_16', 0, 0]]}

#### `to_json()` and `tf.keras.models.model_from_json()`

This is similar to `get_config` / `from_config`, except it turns the model
into a JSON string, which can then be loaded without the original model class.
It is also specific to models, it isn't meant for layers.

**Example:**

In [33]:
model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
json_config = model.to_json()
new_model = keras.models.model_from_json(json_config)

In [34]:
json_config

'{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_5"}}, {"class_name": "Dense", "config": {"name": "dense_17", "trainable": true, "dtype": "float32", "units": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "keras_version": "2.9.0", "backend": "tensorflow"}'

### Custom objects

**Models and layers**

The architecture of subclassed models and layers are defined in the methods
`__init__` and `call`. They are considered Python bytecode,
which cannot be serialized into a JSON-compatible config
-- you could try serializing the bytecode (e.g. via `pickle`),
but it's completely unsafe and means your model cannot be loaded on a different system.

In order to save/load a model with custom-defined layers, or a subclassed model,
you should overwrite the `get_config` and optionally `from_config` methods.
Additionally, you should use register the custom object so that Keras is aware of it.

서브 클래스된 모델과 레이어의 아키텍처는 `__init__` 및 `call` 메서드에 정의되어 있습니다. 그것들은 Python 바이트 코드로 간주되며 JSON-호환 config으로 직렬화될 수 없습니다 -- 바이트 코드 직렬화를 시도할 수는 있지만(예: pickle을 통해) 완전히 불안전하므로 모델을 다른 시스템에 로드할 수 없습니다.

사용자 정의 레이어를 사용하는 모델 또는 서브 클래스된 모델을 저장/로드하려면 get_config 및 선택적으로 from_config 메서드를 override해야 합니다. 또한 Keras가 인식할 수 있도록 사용자 정의 객체를 등록해야 합니다.

**Custom functions**

Custom-defined functions (e.g. activation loss or initialization) do not need a get_config method. The function name is sufficient for loading as long as it is registered as a custom object.

사용자 정의 함수(예: activation loss 또는  initialization)에는 get_config 메서드가 필요하지 않습니다. 함수명은 사용자 정의 객체로 등록되어 있는 한 로드하기에 충분합니다.

**Loading the TensorFlow graph only**

It's possible to load the TensorFlow graph generated by the Keras. If you do so, you won't need to provide any custom_objects. You can do so like this:

Keras가 생성한 TensorFlow 그래프를 로드할 수 있습니다. 그렇게 하면 custom_objects를 제공할 필요가 없습니다. 다음과 같이 해볼 수 있습니다.

In [39]:
!rmdir -p ./my_model

rmdir: failed to remove './my_model': Directory not empty


In [40]:
!ls -al ./my_model/

total 20
drwxr-xr-x 5 root root 4096 Oct 13 02:06 .
drwxr-xr-x 1 root root 4096 Oct 13 01:18 ..
drwxr-xr-x 2 root root 4096 Oct 13 00:16 assets
drwxr-xr-x 2 root root 4096 Oct 13 02:06 .ipynb_checkpoints
drwxr-xr-x 2 root root 4096 Oct 13 01:07 variables


In [41]:
model.save("my_model2")
tensorflow_graph = tf.saved_model.load("my_model2")
x = np.random.uniform(size=(4, 32)).astype(np.float32)
predicted = tensorflow_graph(x).numpy()


In [42]:
print(predicted)

[[-0.83285755]
 [-0.70925194]
 [-0.49494576]
 [-0.9361056 ]]


In [43]:
print(type(tensorflow_graph))

<class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>


Note that this method has several drawbacks:
* For traceability reasons, you should always have access to the custom
objects that were used. You wouldn't want to put in production a model
that you cannot re-create.
* The object returned by `tf.saved_model.load` isn't a Keras model. So it's
not as easy to use. For example, you won't have access to `.predict()` or `.fit()`

Even if its use is discouraged, it can help you if you're in a tight spot,
for example, if you lost the code of your custom objects or have issues
loading the model with `tf.keras.models.load_model()`.

You can find out more in
the [page about `tf.saved_model.load`](https://www.tensorflow.org/api_docs/python/tf/saved_model/load)

이 메서드에는 몇 가지 단점이 있습니다.
* 추적 가능성을 위해 사용된 사용자 정의 객체에 항상 접근할 수 있어야 합니다. 다시 만들 수 없는 모델을 제품에 넣고 싶지 않을 것입니다.
* `tf.saved_model.load`가 리턴한 객체는 Keras 모델이 아닙니다. 따라서 사용하기가 쉽지 않습니다. 예를 들면, `.predict()` 또는 `.fit()`에 접근할 수 없습니다.

사용을 권장하지는 않지만, 사용자 정의 객체의 코드를 잃어버렸거나 tf.keras.models.load_model() 모델을 로드하는 데 문제가 있는 경우와 같이 곤란한 상황에서는 도움이 될 수 있습니다.

#### Defining the config methods

Specifications:

* `get_config` should return a JSON-serializable dictionary in order to be
compatible with the Keras architecture- and model-saving APIs.
* `from_config(config)` (`classmethod`) should return a new layer or model
object that is created from the config.
The default implementation returns `cls(**config)`.

**Example:**

In [ ]:
def showarg(a,b,c):
  print(a,b,c)

p={'a':'1','b':'2','c':'3'}
showarg(*p)
showarg(**p)

In [2]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        print("__init__")
        self.var = tf.Variable(a, name="var_a")

    def call(self, inputs, training=False):
        if training:
            return inputs * self.var
        else:
            return inputs

    def get_config(self):
        print("get_config")
        return {"a": self.var.numpy()}

    # There's actually no need to define `from_config` here, since returning
    # `cls(**config)` is the default behavior.
    @classmethod
    def from_config(cls, config):
        print("from_config")
        return cls(**config)

In [3]:
layer = CustomLayer(5)
layer.var.assign(2)
print("current config:", layer.get_config())
print("1")

__init__
get_config
current config: {'a': 2}
1


In [ ]:
serialized_layer = keras.layers.serialize(layer)

In [5]:
new_layer = keras.layers.deserialize(
    serialized_layer, custom_objects={"CustomLayer": CustomLayer}
)

from_config
__init__


In [6]:
print(new_layer)

In [7]:
print(layer)

In [8]:
new_layer.get_config()

get_config


{'a': 2}

#### Registering the custom object

Keras keeps a note of which class generated the config.
From the example above, `tf.keras.layers.serialize`
generates a serialized form of the custom layer:

Keras는 config를 생성한 클래스를 기록합니다. 위의 예에서 tf.keras.layers.serialize는 사용자 정의 레이어의 직렬화된 형식을 생성합니다.

```
{'class_name': 'CustomLayer', 'config': {'a': 2}}
```

Keras keeps a master list of all built-in layer, model, optimizer,
and metric classes, which is used to find the correct class to call `from_config`.
If the  class can't be found, then an error is raised (`Value Error: Unknown layer`).
There are a few ways to register custom classes to this list:

`from_config`를 호출할 정확한 클래스를 찾는 데 사용되는 모든 빌트인 레이어, 모델, 옵티마이저 및 메트릭 클래스의 마스터 목록을 유지합니다. 클래스를 찾을 수 없으면 오류가 발생합니다(Value Error: Unknown layer). 다음 목록은 사용자 정의 클래스를 등록하는 몇 가지 방법입니다.

1. Setting `custom_objects` argument in the loading function. (see the example
in section above "Defining the config methods")
2. `tf.keras.utils.custom_object_scope` or `tf.keras.utils.CustomObjectScope`
3. `tf.keras.utils.register_keras_serializable`

#### Custom layer and function example

In [9]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config

In [10]:
def custom_activation(x):
    return tf.nn.tanh(x) ** 2


# Make a model with the CustomLayer and custom_activation
inputs = keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = keras.layers.Activation(custom_activation)(x)
model = keras.Model(inputs, outputs)

In [11]:
# Retrieve the config
config = model.get_config()
print(config)

{'name': 'model', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 32), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'}, 'name': 'input_1', 'inbound_nodes': []}, {'class_name': 'CustomLayer', 'config': {'name': 'custom_layer', 'trainable': True, 'dtype': 'float32', 'units': 32}, 'name': 'custom_layer', 'inbound_nodes': [[['input_1', 0, 0, {}]]]}, {'class_name': 'Activation', 'config': {'name': 'activation', 'trainable': True, 'dtype': 'float32', 'activation': 'custom_activation'}, 'name': 'activation', 'inbound_nodes': [[['custom_layer', 0, 0, {}]]]}], 'input_layers': [['input_1', 0, 0]], 'output_layers': [['activation', 0, 0]]}


In [12]:
# At loading time, register the custom objects with a `custom_object_scope`:
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.Model.from_config(config)

In [14]:
model.get_config()

{'name': 'model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_1'},
   'name': 'input_1',
   'inbound_nodes': []},
  {'class_name': 'CustomLayer',
   'config': {'name': 'custom_layer',
    'trainable': True,
    'dtype': 'float32',
    'units': 32},
   'name': 'custom_layer',
   'inbound_nodes': [[['input_1', 0, 0, {}]]]},
  {'class_name': 'Activation',
   'config': {'name': 'activation',
    'trainable': True,
    'dtype': 'float32',
    'activation': 'custom_activation'},
   'name': 'activation',
   'inbound_nodes': [[['custom_layer', 0, 0, {}]]]}],
 'input_layers': [['input_1', 0, 0]],
 'output_layers': [['activation', 0, 0]]}

In [13]:
new_model.get_config()

{'name': 'model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_1'},
   'name': 'input_1',
   'inbound_nodes': []},
  {'class_name': 'CustomLayer',
   'config': {'name': 'custom_layer',
    'trainable': True,
    'dtype': 'float32',
    'units': 32},
   'name': 'custom_layer',
   'inbound_nodes': [[['input_1', 0, 0, {}]]]},
  {'class_name': 'Activation',
   'config': {'name': 'activation',
    'trainable': True,
    'dtype': 'float32',
    'activation': 'custom_activation'},
   'name': 'activation',
   'inbound_nodes': [[['custom_layer', 0, 0, {}]]]}],
 'input_layers': [['input_1', 0, 0]],
 'output_layers': [['activation', 0, 0]]}

### In-memory model cloning

You can also do in-memory cloning of a model via `tf.keras.models.clone_model()`.
This is equivalent to getting the config then recreating the model from its config
(so it does not preserve compilation information or layer weights values).

**Example:**

tf.keras.models.clone_model()을 통해 특정 모델에 대한 인메모리 clone을 수행할 수도 있습니다. 이는 config를 얻은 다음 이 config에서 모델을 다시 생성하는 것과 같습니다(따라서 컴파일 정보 또는 레이어 weights 값을 유지하지 않습니다).

In [15]:
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.models.clone_model(model)

## Saving & loading only the model's weights values

You can choose to only save & load a model's weights. This can be useful if:

- You only need the model for inference: in this case you won't need to
restart training, so you don't need the compilation information or optimizer state.
- You are doing transfer learning: in this case you will be training a new model
reusing the state of a prior model, so you don't need the compilation
information of the prior model.

모델의 weight 값들만 저장하고 로드하도록 선택할 수 있습니다. 다음과 같은 경우에 유용할 수 있습니다.
* inference(predict)을 위한 모델만 필요합니다: 이 경우 training을 다시 시작할 필요가 없으므로 컴파일 정보나 옵티마이저 상태가 필요하지 않습니다.
* 전이 학습을 수행하고 있습니다: 이 경우 이전 모델의 상태를 재사용하는 새 모델을 훈련하므로 이전 모델의 컴파일 정보가 필요하지 않습니다.(Pre-Trained Model의 Backbone Network은 training하지 않기 때문에)

### APIs for in-memory weight transfer

Weights can be copied between different objects by using `get_weights`
and `set_weights`:

* `tf.keras.layers.Layer.get_weights()`: Returns a list of numpy arrays.
* `tf.keras.layers.Layer.set_weights()`: Sets the model weights to the values
in the `weights` argument(이 함수의 `weights` 인수의 값으로 모델의 weights을 설정합니다)

Examples below.


***Transfering weights from one layer to another, in memory***

In [16]:
def create_layer():
    layer = keras.layers.Dense(64, activation="relu", name="dense_2")
    layer.build((None, 784))
    
    return layer


layer_1 = create_layer()
layer_2 = create_layer()

# Copy weights from layer 1 to layer 2
layer_2.set_weights(layer_1.get_weights())

***Transfering weights from one model to another model with a
compatible architecture, in memory***

In [17]:
# Create a simple functional model
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# Define a subclassed model with the same architecture
class SubclassedModel(keras.Model):
    def __init__(self, output_dim, name=None):
        super(SubclassedModel, self).__init__(name=name)
        self.output_dim = output_dim
        self.dense_1 = keras.layers.Dense(64, activation="relu", name="dense_1")
        self.dense_2 = keras.layers.Dense(64, activation="relu", name="dense_2")
        self.dense_3 = keras.layers.Dense(output_dim, name="predictions")

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x

    def get_config(self):
        return {"output_dim": self.output_dim, "name": self.name}


subclassed_model = SubclassedModel(10)
# Call the subclassed model once to create the weights.
subclassed_model(tf.ones((1, 784)))

# Copy weights from functional_model to subclassed_model.
subclassed_model.set_weights(functional_model.get_weights())

assert len(functional_model.weights) == len(subclassed_model.weights)
for a, b in zip(functional_model.weights, subclassed_model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

In [ ]:
functional_model.weights

***The case of stateless layers***

Because stateless layers do not change the order or number of weights,
models can have compatible architectures even if there are extra/missing
stateless layers.

상태 비저장 레이어는 가중치의 순서나 수를 변경하지 않기 때문에 상태 비저장 레이어가 추가되거나 누락된 경우에도 모델은 호환 가능한 아키텍처를 가질 수 있습니다.

In [18]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

In [19]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
# Add a dropout layer, which does not contain any weights.
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, name="predictions")(x)

functional_model_with_dropout = keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

In [20]:
functional_model_with_dropout.set_weights(functional_model.get_weights())

### APIs for saving weights to disk & loading them back

Weights can be saved to disk by calling `model.save_weights`
in the following formats:

* TensorFlow Checkpoint
* HDF5

The default format for `model.save_weights` is TensorFlow checkpoint.
There are two ways to specify the save format:

1. `save_format` argument: Set the value to `save_format="tf"` or `save_format="h5"`.
2. `path` argument: If the path ends with `.h5` or `.hdf5`,
then the HDF5 format is used. Other suffixes will result in a TensorFlow
checkpoint unless `save_format` is set.

There is also an option of retrieving weights as in-memory numpy arrays.
Each API has its pros and cons which are detailed below.

### TF Checkpoint format

**Example:**

In [21]:
# Runnable example
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("ckpt")
load_status = sequential_model.load_weights("ckpt")

# `assert_consumed` can be used as validation that all variable values have been
# restored from the checkpoint. See `tf.train.Checkpoint.restore` for other
# methods in the Status object.
load_status.assert_consumed()

#### Format details

The TensorFlow Checkpoint format saves and restores the weights using
object attribute names. For instance, consider the `tf.keras.layers.Dense` layer.
The layer contains two weights: `dense.kernel` and `dense.bias`.
When the layer is saved to the `tf` format, the resulting checkpoint contains the keys
`"kernel"` and `"bias"` and their corresponding weight values.
For more information see
["Loading mechanics" in the TF Checkpoint guide](https://www.tensorflow.org/guide/checkpoint#loading_mechanics).

Note that attribute/graph edge is named after **the name used in parent object,
not the name of the variable**. Consider the `CustomLayer` in the example below.
The variable `CustomLayer.var` is saved with `"var"` as part of key, not `"var_a"`.

attribute/graph edge는 변수명이 아니라 부모 객체에서 사용된 이름에 따라 이름이 지정됩니다. 아래 예제의 CustomLayer를 고려해 봅시다. 변수 CustomLayer.var는 "var_a"가 아니라, key의 일부로서 "var"로 저장됩니다.

In [22]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")


layer = CustomLayer(5)
layer_ckpt = tf.train.Checkpoint(layer=layer).save("custom_layer")

ckpt_reader = tf.train.load_checkpoint(layer_ckpt)

ckpt_reader.get_variable_to_dtype_map()

{'save_counter/.ATTRIBUTES/VARIABLE_VALUE': tf.int64,
 '_CHECKPOINTABLE_OBJECT_GRAPH': tf.string,
 'layer/var/.ATTRIBUTES/VARIABLE_VALUE': tf.int32}

#### Transfer learning example

Essentially, as long as two models have the same architecture,
they are able to share the same checkpoint.

**Example:**

In [23]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# Extract a portion of the functional model defined in the Setup section.
# The following lines produce a new model that excludes the final output
# layer of the functional model.
pretrained = keras.Model(
    functional_model.inputs, functional_model.layers[-1].input, name="pretrained_model"
)

In [ ]:
functional_model.summary()

In [ ]:
functional_model.layers[-1].get_config()

In [ ]:
functional_model.layers[-1].input

In [ ]:
functional_model.layers[-1].get_config()

In [36]:
pretrained.summary()

Model: "pretrained_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pretrained.layers[2].get_config()

In [ ]:
# Randomly assign "trained" weights.
for w in pretrained.weights:
    w.assign(tf.random.normal(w.shape))
pretrained.save_weights("pretrained_ckpt")
pretrained.summary()

In [38]:
# Assume this is a separate program where only 'pretrained_ckpt' exists.
# Create a new functional model with a different output dimension.
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
### backbone network

# training
#classification layer + softmax
outputs = keras.layers.Dense(5, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="new_model")

# Load the weights from pretrained_ckpt into model.
model.load_weights("pretrained_ckpt")

# Check that all of the pretrained weights have been loaded.
for a, b in zip(pretrained.weights, model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

print("\n", "-" * 50)
model.summary()


 --------------------------------------------------
Model: "new_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 predictions (Dense)         (None, 5)                 325       
                                                                 
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Example 2: Sequential model
# Recreate the pretrained model, and load the saved weights.
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
pretrained_model = keras.Model(inputs=inputs, outputs=x, name="pretrained")

# Sequential example:
model = keras.Sequential([pretrained_model, keras.layers.Dense(5, name="predictions")])
model.summary()

pretrained_model.load_weights("pretrained_ckpt")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pretrained (Functional)     (None, 64)                54400     
                                                                 
 predictions (Dense)         (None, 5)                 325       
                                                                 
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.summary()
# Warning! Calling `model.load_weights('pretrained_ckpt')` won't throw an error,
# but will *not* work as expected. If you inspect the weights, you'll see that
# none of the weights will have loaded. `pretrained_model.load_weights()` is the
# correct method to call.
model.load_weight("pretrained_ckpt")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pretrained (Functional)     (None, 64)                54400     
                                                                 
 predictions (Dense)         (None, 5)                 325       
                                                                 
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________


AttributeError: ignored

It is generally recommended to stick to the same API for building models. If you
switch between Sequential and Functional, or Functional and subclassed,
etc., then always rebuild the pre-trained model and load the pre-trained
weights to that model.

The next question is, how can weights be saved and loaded to different models
if the model architectures are quite different?
The solution is to use `tf.train.Checkpoint` to save and restore the exact layers/variables.

**Example:**

일반적으로 모델을 빌드할 때 동일한 API를 사용하는 것이 좋습니다. Sequential 및 Functional 또는 Functional 및 서브 클래스 등 간에 전환하는 경우, 항상 사전 훈련된 모델을 다시 빌드하고 사전 훈련된 가중치를 해당 모델에 로드합니다.

다음 질문은 모델 아키텍처가 상당히 다른 경우 어떻게 다른 모델에 가중치를 저장하고 로드하는가입니다. 해결책은 tf.train.Checkpoint를 사용하여 정확한 레이어/변수를 저장하고 복원하는 것입니다

In [43]:
functional_model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 predictions (Dense)         (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [48]:
# The next question is, how can weights be saved and loaded to different models 
# if the model architectures are quite different? 
# The solution is to use tf.train.Checkpoint to save and restore the exact layers/variables.

# Create a subclassed model that essentially uses functional_model's first
# and last layers.
# First, save the weights of functional_model's first and last dense layers.
first_dense = functional_model.layers[1] #dense_1 (Dense)             (None, 64)
last_dense = functional_model.layers[-1] #predictions (Dense)         (None, 10)
ckpt_path = tf.train.Checkpoint(
    dense=first_dense, kernel=last_dense.kernel, bias=last_dense.bias
).save("./dfckpt/ckpt")

In [49]:
# Define the subclassed model.
class ContrivedModel(keras.Model):
    def __init__(self):
        super(ContrivedModel, self).__init__()
        self.first_dense = keras.layers.Dense(64)
        self.kernel = self.add_variable("kernel", shape=(64, 10))
        self.bias = self.add_variable("bias", shape=(10,))

    def call(self, inputs):
        x = self.first_dense(inputs)
        return tf.matmul(x, self.kernel) + self.bias

In [50]:
model = ContrivedModel()
# Call model on inputs to create the variables of the dense layer.
_ = model(tf.ones((1, 784)))

# Create a Checkpoint with the same structure as before, and load the weights.
tf.train.Checkpoint(
    dense=model.first_dense, kernel=model.kernel, bias=model.bias
).restore(ckpt_path).assert_consumed()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  import sys


### HDF5 format

The HDF5 format contains weights grouped by layer names.
The weights are lists ordered by concatenating the list of trainable weights
to the list of non-trainable weights (same as `layer.weights`).
Thus, a model can use a hdf5 checkpoint if it has the same layers and trainable
statuses as saved in the checkpoint.

**Example:**

HDF5 형식에는 레이어 이름별로 그룹화된 가중치가 포함됩니다. 가중치는 훈련 가능한 가중치 목록을 훈련 불가능한 가중치 목록(layer.weights와 동일)에 연결하여 정렬된 목록입니다. 따라서 모델이 체크포인트에 저장된 것과 동일한 레이어 및 훈련 가능한 상태를 갖는 경우 hdf5 체크포인트을 사용할 수 있습니다.

예제:

In [51]:
# Runnable example
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("weights.h5")
sequential_model.load_weights("weights.h5")

Note that changing `layer.trainable` may result in a different
`layer.weights` ordering when the model contains nested layers.

모델에 중첩된 레이어가 포함된 경우 layer.trainable을 변경하면 layer.weights의 순서가 다르게 나타날 수 있습니다.

In [52]:
class NestedDenseLayer(keras.layers.Layer):
    def __init__(self, units, name=None):
        super(NestedDenseLayer, self).__init__(name=name)
        self.dense_1 = keras.layers.Dense(units, name="dense_1")
        self.dense_2 = keras.layers.Dense(units, name="dense_2")

    def call(self, inputs):
        return self.dense_2(self.dense_1(inputs))


nested_model = keras.Sequential([keras.Input((784,)), NestedDenseLayer(10, "nested")])
variable_names = [v.name for v in nested_model.weights]
print("variables: {}".format(variable_names))

print("\nChanging trainable status of one of the nested layers...")
nested_model.get_layer("nested").dense_1.trainable = False

variable_names_2 = [v.name for v in nested_model.weights]
print("\nvariables: {}".format(variable_names_2))
print("variable ordering changed:", variable_names != variable_names_2)

variables: ['nested/dense_1/kernel:0', 'nested/dense_1/bias:0', 'nested/dense_2/kernel:0', 'nested/dense_2/bias:0']

Changing trainable status of one of the nested layers...

variables: ['nested/dense_2/kernel:0', 'nested/dense_2/bias:0', 'nested/dense_1/kernel:0', 'nested/dense_1/bias:0']
variable ordering changed: True


#### Transfer learning example

When loading pretrained weights from HDF5, it is recommended to load
the weights into the original checkpointed model, and then extract
the desired weights/layers into a new model.

**Example:**

전이 학습 예제
HDF5에서 사전 훈련된 가중치를 로딩할 때는 가중치를 기존 체크포인트 모델에 로드한 다음 원하는 가중치/레이어를 새 모델로 추출하는 것이 좋습니다.

In [53]:
def create_functional_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = keras.layers.Dense(10, name="predictions")(x)
    return keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")


functional_model = create_functional_model()
functional_model.save_weights("pretrained_weights.h5")



In [55]:
# In a separate program:
pretrained_model = create_functional_model()
pretrained_model.load_weights("pretrained_weights.h5")
pretrained_model.summary()


Model: "3_layer_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 predictions (Dense)         (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [59]:
xxx = pretrained_model.layers[:-1]
# print(xxx)
print(xxx[0].get_config())
print(xxx[1].get_config())
print(xxx[2].get_config())

{'batch_input_shape': (None, 784), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'digits'}
{'name': 'dense_1', 'trainable': True, 'dtype': 'float32', 'units': 64, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dense_2', 'trainable': True, 'dtype': 'float32', 'units': 64, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}


In [60]:
# Create a new model by extracting layers from the original model:
extracted_layers = pretrained_model.layers[:-1]
extracted_layers.append(keras.layers.Dense(5, name="dense_3"))
model = keras.Sequential(extracted_layers)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________
